In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import os
import json
import time
from datetime import datetime

# ------------------------- Helper Functions ------------------------- #

def save_full_screenshot(driver, regnumber=""):
    """Take a screenshot and save it with timestamp."""
    try:
        os.makedirs("screenshots", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"screenshots/{regnumber}_{timestamp}.png"
        driver.save_screenshot(filename)
        print(f"📸 Screenshot saved: {filename}")
    except Exception as e:
        print(f"❌ Screenshot failed: {e}")


def init_database():
    """Ensure database.json exists and has correct structure."""
    if not os.path.exists("database.json"):
        data = {"bids": [], "status": "", "current_bid": ""}
        with open("database.json", "w") as f:
            json.dump(data, f, indent=4)
    else:
        with open("database.json", "r") as f:
            data = json.load(f)
        if isinstance(data, list) or not isinstance(data, dict):
            data = {"bids": [], "status": "", "current_bid": ""}
            with open("database.json", "w") as f:
                json.dump(data, f, indent=4)
    return data



def monitor_bids(driver, reg="reg"):

    if not driver:
        print("Driver not found!")
        return

    data = init_database()

    while True:
        try:
            driver.current_url 
        except:
            print("🛑 Browser closed manually... stopping script!")
            break


        driver.refresh()
        time.sleep(2)


        try:
            try:
                time_left = driver.find_element(By.ID, "cd").text.strip()  # e.g., "28m 20s"
            except:
                time_left = ""

            tl = time_left.lower()

            # Stop only when truly ended
            if (
                not tl or                  # blank / no time
                "ended" in tl or
                "closed" in tl or
                tl == "0s" or
                tl == "0m 0s" or
                tl.startswith("0m")        # 0m Xs (any second)
            ):
                print(f"⛔ Auction Ended or Time Up ({time_left}). Stopping...")
                break
        except:
            pass

        try:
            current_bid = driver.find_element(By.ID, "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblCurrentBid").text
            current_bid = current_bid.replace("£", "").replace(",", "").strip()
        except:
            current_bid = None


        try:
            status = driver.find_element(By.ID, "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblReserveNotMet").text.strip()
        except:
            status = ""


        bid_history = []
        try:
            view_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lnkViewBids"))
            )
            driver.execute_script("arguments[0].click();", view_btn)
            time.sleep(1)

            bids_elements = driver.find_elements(
                By.XPATH,
                '//table[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_ucViewBids_gdvBids"]//tr/td[2]/span'
            )
            bid_history = [b.text.replace("£", "").replace(",", "").strip() for b in bids_elements]

  
            try:
                close_btn = driver.find_element(By.ID, "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_btnCloseViewBids")
                driver.execute_script("arguments[0].click();", close_btn)
            except:
                pass
        except:
            pass


        updated = False
        if data.get("status") != status:
            data["status"] = status
            updated = True
            print(f"📝 Status Updated: {status}")

        if data.get("current_bid") != current_bid:
            data["current_bid"] = current_bid
            updated = True
            print(f"💵 Current Bid Updated: {current_bid}")


        for b in bid_history:
            if b not in data["bids"]:
                data["bids"].append(b)
                updated = True
                print(f"💰 New Bid Added: {b}")

        if updated:
            with open("database.json", "w") as f:
                json.dump(data, f, indent=4)

        time.sleep(10)



def scrape(path, username="Mohsin7865", password="Muhssan5687"):
    options = ChromeOptions()
    options.headless = False
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()


    try:
        cookie = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
        )
        cookie.click()
    except:
        pass


    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_txtUsername"))
        ).send_keys(username)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_txtPassword"))
        ).send_keys(password)
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "ContentPlaceHolder1_btnLogin"))
        ).click()
        print("🔐 Login Successful")
    except Exception as e:
        print("⚠ Login failed:", e)
        return

    time.sleep(5)


    try:
        reg = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//span[contains(@id,"lblRegNo")]'))
        ).text.strip()
    except:
        reg = "reg_not_found"


    folder = "live_html"
    os.makedirs(folder, exist_ok=True)
    filename = os.path.join(folder, f"{reg}.html")
    with open(filename, "w", encoding="utf-8") as f:
        f.write(driver.execute_script("return document.documentElement.outerHTML;"))


    init_database()


    monitor_bids(driver, reg)



if __name__ == "__main__":
    path = "https://www.agnewtradecentre.com/vehicle.aspx?id=129622"
    scrape(path)


🔐 Login Successful
📝 Status Updated: (Reserve Not Met)
💵 Current Bid Updated: 16300
💰 New Bid Added: 16300
💰 New Bid Added: 16200
💰 New Bid Added: 16100
💰 New Bid Added: 16000
💰 New Bid Added: 14900
💰 New Bid Added: 14800
🛑 Browser closed manually... stopping script!


In [2]:
from bs4 import BeautifulSoup
import os,json,re
import pandas as pd

def extract_vehicle_info(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    result = {}

    ids = [
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblModel",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblVariant",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblRegNo",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblRegDate",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblMileage",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblColour",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblTransmissionType",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblFuelType",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblCO2",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblDoors",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblAuctionStartDate",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblAuctionEndDate",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblTradeNo",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblLotNumber",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblDeliveryCharge",
        "ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblDeliveryNote"
    ]

    for id_ in ids:
        span = soup.find("span", id=id_)
        result[id_] = span.text.strip() if span else None

    return result

def scrap_by_html_to_csv(folder="live_html", output_csv="Agnewtradecentre.csv"):
    if not os.path.exists(folder):
        print(f"Folder '{folder}' does not exist!")
        return

    files = sorted(os.listdir(folder))
    if not files:
        print("No HTML files found in folder.")
        return

    all_data = []  

    for file in files:
        if not file.endswith(".html"):
            continue
        path = os.path.join(folder, file)
        with open(path, "r", encoding="utf-8") as f:
            html = f.read()
            soup = BeautifulSoup(html, "html.parser")

            title_span = soup.find("span", id="ContentPlaceHolder1_ContentSearch_lblTitle")
            title = title_span.text.strip() if title_span else "Title Not Found"

            values = extract_vehicle_info(html)
            dor=values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblRegDate"]
            start_dt = values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblAuctionStartDate"]
            end_dt   = values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblAuctionEndDate"]
            start_date = start_dt.split()[0] + " " + start_dt.split()[1] + " " + start_dt.split()[2] if start_dt else ""
            start_time = start_dt.split()[3] if start_dt else ""

            end_date = end_dt.split()[0] + " " + end_dt.split()[1] + " " + end_dt.split()[2] if end_dt else ""
            end_time = end_dt.split()[3] if end_dt else ""
            
            
            
            generalCondition = {}
            interior = {}
            splat_img_td = soup.find("img", id="ContentPlaceHolder1_ContentSearch_tbcSelling_TabPanel2_imgCarSplat")
            if splat_img_td:
                next_td = splat_img_td.find_parent("td").find_next_sibling("td")
                if next_td:
                    table = next_td.find("table")
                    if table:
                        for row in table.find_all("tr"):
                            cols = row.find_all("td")
                            if len(cols) == 2:
                                key = cols[0].text.strip().replace(":", "")
                                value = cols[1].text.strip()
                                interior[key] = value

            tyres = {}
            tyres_b = soup.find("b", string="Tyres")
            if tyres_b:
                tr = tyres_b.find_parent("tr")          
                next_tr = tr.find_next_sibling("tr")    
                if next_tr:
                    td = next_tr.find("td")
                    if td:
                        table = td.find("table")
                        if table:
                            for row in table.find_all("tr"):
                                cols = row.find_all("td")
                                if len(cols) == 2:
                                    key = cols[0].text.strip().replace(":", "")
                                    value = cols[1].text.strip()
                                    tyres[key] = value

            tyres_str = ",".join([f"{k}:{v}" for k, v in tyres.items()])


            generalCondition["Interior"] = interior
   
            generalCondition["Other Information"] = {
                "CO2": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblCO2"],
                "Delivery Charge": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblDeliveryCharge"],
                "Trade No": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblTradeNo"]
            }
            base_url = "https://www.agnewtradecentre.com/"
            Damaged_images = ""
            img_tag = soup.find("img", id="ContentPlaceHolder1_ContentSearch_tbcSelling_TabPanel2_imgCarSplat")
            if img_tag:
                img_src = img_tag.get("src", "")
                if img_src:
                    if img_src.startswith("http"):
                        full_url = img_src
                    else:
                        full_url = base_url + img_src.lstrip("/")
                    Damaged_images = full_url
            
            images_div = soup.find("div", id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_rptImageList_divGallery_0")

            all_images = []
            if images_div:
                table = images_div.find("table")
                if table:
                    links = table.find_all("a", href=True)
                    for link in links:
                        href = link['href'].strip()
                        if href.startswith("http"):
                            all_images.append(href)
                        else:
                            all_images.append(base_url + href.lstrip("/"))


            images_str = ",".join(all_images)

            
            
            CAP_Clean = ""
            CAP_Retail = ""
            Glass = ""

            table2 = soup.find("table", id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_ucGuidePricing_tblGuidePricing")
            if table2:
                rows = table2.find_all("tr")
                if len(rows) >= 3:
                    td_values = rows[2].find_all("td")
                    if len(td_values) >= 3:
                        CAP_Clean = td_values[0].get_text(strip=True)
                        CAP_Retail = td_values[1].get_text(strip=True)
                        Glass = td_values[2].get_text(strip=True)
                        if Glass in ["N/A", "n/a"]:
                            Glass = ""  
                            
            service_history_str = ""
            last_service_mileage = ""
            last_service_date = ""
            total_services = 0

            service_table = soup.find("table", id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabAdditionalInfo_gdvServiceHistory")
            if service_table:
                rows = service_table.find_all("tr")
                total_services = len(rows) - 1
                for row in rows[1:]:  
                    cols = row.find_all("td")
                    if len(cols) >= 4:
                        entry = f"{cols[3].text.strip()} {cols[1].text.strip()} {cols[2].text.strip()}"
                        service_history_str += entry + ","
                
          
                if total_services > 0:
                    last_row = rows[-1]
                    last_cols = last_row.find_all("td")
                    last_service_date = last_cols[1].text.strip()
                    last_service_mileage = last_cols[2].text.strip()

     
            service_history_str = service_history_str.rstrip(",")
            
            
            MOT_Expiry = ""

            Keys = ""
            No_of_Registered_Keepers = ""
            CO2_Emissions = ""
            Seller = ""


            additional_info_table = soup.find("div", id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabAdditionalInfo").find("table")
            if additional_info_table:
                for row in additional_info_table.find_all("tr"):
                    cols = row.find_all("td")
                    if len(cols) == 2:
                        key = cols[0].text.strip()
                        value = cols[1].text.strip()
                        

                        if "MOT Expiry" in key:
                            MOT_Expiry = value
                        elif "Keys" in key:
                            Keys = value
                        elif "Registered Keepers" in key:
                            No_of_Registered_Keepers = value
                 
                        elif "C02 Emissions" in key or "CO2 Emissions" in key:
                            CO2_Emissions = value
                        elif "Seller" in key:
                            Seller = value


            additional_spec_json = {"Additional Specification": {}}


            additional_div = soup.find("div", id="ContentPlaceHolder1_ContentSearch_tbcSelling_TabPanel1_divDFA")
            if additional_div:
                items = additional_div.find_all("li")
                for idx, li in enumerate(items, start=1):
                    key = f"Note {idx}"
                    value = li.text.strip()
                    additional_spec_json["Additional Specification"][key] = value

            json_filepath="database.json"
            with open (json_filepath,"r",encoding="utf-8") as f:
                data  = json.load(f)
            bids = data.get("bids", [])
            status = data.get("status", "")
            current_bid = data.get("current_bid", "")   
            status_clean = re.sub(r"\(.*?\)", "", status).strip() 
            row = {
                "Title": title,
                "Make": title.split()[0] if title else "",
                "Model": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblModel"],
                "Variant": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblVariant"],
                "Reg": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblRegNo"],
                "D.O.R": dor,
                "Year": dor.split("/")[-1] if dor else "",
                "mileage": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblMileage"],
                "Colour": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblColour"],
                "Transmission": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblTransmissionType"],
                "Fuel Type": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblFuelType"],
                "Doors": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblDoors"],
                "Start Date": start_date,
                "Start Time": start_time,
                "End Date": end_date,
                "End Time": end_time,
                "Lot": values["ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblLotNumber"],
                "General Condition": generalCondition,
                "Tyres Condition":tyres_str,
                "Damaged_images":Damaged_images,
                "Images":images_str,
                "CAP Clean":CAP_Clean,
                "CAP Retail":CAP_Retail,
                "Glass Retail":Glass,
                "Service History":service_history_str,
                "No of services":total_services,
                "Last Service":last_service_date,
                "Last service mileage":last_service_mileage,
                "MOT Expiry Date":MOT_Expiry,
                "Keys":Keys,
                "Former Keepers":No_of_Registered_Keepers,
                "Vendor":Seller,
                "Additional information":additional_spec_json,
                "Bidding History":bids,
                "Last Bid":current_bid,
                "Bidding Status":status,
            }

            all_data.append(row)

    df = pd.DataFrame(all_data)
    df.to_csv(output_csv, index=False, encoding="utf-8")
    print(f"✅ All HTML files saved to '{output_csv}'")

scrap_by_html_to_csv()


✅ All HTML files saved to 'Agnewtradecentre.csv'


In [3]:
from urllib.parse import urlparse, urljoin
import threading, requests, os, re
import pandas as pd
from PIL import Image, ImageDraw, ImageFont


df = pd.read_csv("Agnewtradecentre.csv")


reg_img = df[['Reg', "Images", "Damaged_images"]]



def add_watermark_to_image(image_path, text="Sourced from Agnew Trade Centre"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        try:
            font = ImageFont.truetype("arial.ttf", 10)
        except:
            font = ImageFont.load_default()

        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        tw, th = bbox[2] - bbox[0], bbox[3] - bbox[1]
        x, y = image.width - tw - margin, image.height - th - margin

        draw.rectangle([x - margin, y - margin, x + tw + margin, y + th + margin], fill=(0,0,0,160))
        draw.text((x, y), text, font=font, fill=(255,255,255,200))
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"✔ Watermarked: {image_path}")

    except Exception as e:
        print(f"⚠ Watermark Error: {e}")


def download_images(data, main_folder="Images"):
    os.makedirs(main_folder, exist_ok=True)

    for index, row in data.iterrows():
        reg_no = str(row["Reg"]).strip()

    
        img_urls = [u for u in re.split(r',\s*', str(row["Images"])) if u]
        dmg_urls = [u for u in re.split(r',\s*', str(row["Damaged_images"])) if u]
        
        
        reg_folder = os.path.join(main_folder, reg_no)
        original_folder = os.path.join(reg_folder, "Original")
        damage_folder = os.path.join(reg_folder, "Damage")

        os.makedirs(original_folder, exist_ok=True)
        os.makedirs(damage_folder, exist_ok=True)

        def save_img(url, folder, idx):
            url = url.strip()
            if not url:
                return

            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url)

            parsed = urlparse(url)
            if not parsed.netloc:
                print(f"❌ Invalid URL Skipped: {url}")
                return

            full_path = os.path.join(folder, f"{reg_no}_{idx}.jpg")


            if os.path.exists(full_path):
                print(f"⏩ Skipped (Exists): {full_path}")
                return

            try:
                response = requests.get(url, stream=True, timeout=20)
                response.raise_for_status()

                with open(full_path, "wb") as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                add_watermark_to_image(full_path)
                print(f"📌 Saved: {full_path}")

            except Exception as e:
                print(f"⚠ Error downloading: {url} -> {e}")

        for i, url in enumerate(img_urls):
            save_img(url, original_folder, i+1)


        for i, url in enumerate(dmg_urls):
            save_img(url, damage_folder, i+1)



def start_funcs():
    t1 = threading.Thread(target=download_images, args=(reg_img,))
   

    t1.start()
    t1.join() 
 
if __name__ == "__main__":
    start_funcs()


✔ Watermarked: Images\KX22DZE\Original\KX22DZE_1.jpg
📌 Saved: Images\KX22DZE\Original\KX22DZE_1.jpg
✔ Watermarked: Images\KX22DZE\Original\KX22DZE_2.jpg
📌 Saved: Images\KX22DZE\Original\KX22DZE_2.jpg
✔ Watermarked: Images\KX22DZE\Original\KX22DZE_3.jpg
📌 Saved: Images\KX22DZE\Original\KX22DZE_3.jpg
✔ Watermarked: Images\KX22DZE\Original\KX22DZE_4.jpg
📌 Saved: Images\KX22DZE\Original\KX22DZE_4.jpg
✔ Watermarked: Images\KX22DZE\Original\KX22DZE_5.jpg
📌 Saved: Images\KX22DZE\Original\KX22DZE_5.jpg
✔ Watermarked: Images\KX22DZE\Original\KX22DZE_6.jpg
📌 Saved: Images\KX22DZE\Original\KX22DZE_6.jpg
✔ Watermarked: Images\KX22DZE\Original\KX22DZE_7.jpg
📌 Saved: Images\KX22DZE\Original\KX22DZE_7.jpg
✔ Watermarked: Images\KX22DZE\Original\KX22DZE_8.jpg
📌 Saved: Images\KX22DZE\Original\KX22DZE_8.jpg
✔ Watermarked: Images\KX22DZE\Original\KX22DZE_9.jpg
📌 Saved: Images\KX22DZE\Original\KX22DZE_9.jpg
✔ Watermarked: Images\KX22DZE\Damage\KX22DZE_1.jpg
📌 Saved: Images\KX22DZE\Damage\KX22DZE_1.jpg
